In [1]:
trace_directory = '~/.ros/tracing/profile'

In [2]:
import sys
# Assuming a workspace with:
#   src/tracetools_analysis/
#   src/micro-ROS/ros_tracing/ros2_tracing/tracetools_read/
sys.path.insert(0, '../')
sys.path.insert(0, '../../../micro-ROS/ros_tracing/ros2_tracing/tracetools_read/')
import datetime as dt
import os

from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import PrintfTickFormatter
import numpy as np
import pandas as pd

from tracetools_analysis import utils
from tracetools_analysis.conversion import ctf
from tracetools_analysis.loading import load_pickle
from tracetools_analysis.processor import Processor
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.processor.profile import ProfileHandler

In [3]:
# Convert
trace_directory = os.path.expanduser(trace_directory)
pickle_filename = 'pickle'
pickle_path = os.path.join(trace_directory, pickle_filename)
count = ctf.convert(trace_directory, pickle_path)
print(f'{count} events')

# Process
events = load_pickle(pickle_path)
profile_handler = ProfileHandler()
ros_handler = Ros2Handler()
processor = Processor(profile_handler, ros_handler)
processor.process(events)
data_model = profile_handler.get_data_model()
#data_model.print_model()

50832 events


In [9]:
data_util = utils.ProfileDataModelUtil(data_model)
ros_data_util = utils.RosDataModelUtil(ros_handler.get_data_model())

output_notebook()
psize = 450

# Plot durations
for tid in list(data_util.get_tids()):
    node_names = ros_data_util.get_node_names_from_tid(tid)
    node_names_format = ', '.join(node_names) if node_names is not None else 'unknown'
    functions_data = data_util.get_function_duration_data(tid)
    for function_data in functions_data:
        depth = function_data['depth']
        name = function_data['function_name']
        parent = function_data['parent_name']
        df = function_data['data']

        # Duration vs. actual duration
        starttime = dt.datetime.fromtimestamp(df.loc[:, 'start_timestamp'].iloc[0] / (1000.0 * 1000000.0)).strftime('%Y-%m-%d %H:%M')
        duration = df[['start_timestamp', 'duration']]
        actual_duration = df[['start_timestamp', 'actual_duration']]
        duration_source = ColumnDataSource(duration)
        actual_duration_source = ColumnDataSource(actual_duration)
        duration = figure(
            title=f'Function duration over time: {name}',
            x_axis_label=f'start ({starttime})',
            y_axis_label='duration (ms)',
            plot_width=psize, plot_height=psize)
        duration.title.align = 'center'
        duration.line(
            x='start_timestamp',
            y='duration',
            color='red',
            legend='effective',
            line_width=2,
            source=duration_source)
        duration.line(
            x='start_timestamp',
            y='actual_duration',
            color='blue',
            legend='real',
            line_width=2,
            source=actual_duration_source)
        duration.legend.label_text_font_size = '11px'
        duration.xaxis[0].formatter = DatetimeTickFormatter(seconds=['%Ss'])

        show(duration)

Loading BokehJS ...